### Парсинг данных и создание датафрейма

In [1]:
import os
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

# %matplotlib notebook

In [2]:
runN = "*" # If take all runs set runN = "*"
paths = glob.glob(f"build/Run{runN}_nt_ParticleData_t*.csv")

In [3]:
with open(paths[0], 'r') as f:
    print(f"Headers from file: {paths[0]}")
    
#     lines = f.readline().strip()
#     print(lines)
#     columns = lines[1:]
#     columns = columns.split(",")
#     meta_len = 1

    hlines = []
    while True:
        line = f.readline()
        if len(line) == 0 or line[0] != '#':
            break
        hlines.append(line)
    title = hlines[1].replace('#title ', '')[:-1]
    sep = chr(int(hlines[2].replace('#separator ', '')[:-1]))
    vsc_sep = chr(int(hlines[3].replace('#vector_separator ', '')[:-1]))
    columns = [' '.join(l.split(' ')[2:])[:-1] for l in hlines[4:]]
    meta_len = len(hlines)
    print(f'Metarows: {meta_len}\nColumns: {columns}')

Headers from file: build/Run0_nt_ParticleData_t3.csv
Metarows: 8
Columns: ['particle_name', 'creator_process', 'parentID', 'energy']


In [11]:
data = pd.DataFrame()
for path in paths:
    item = pd.read_csv(path, skiprows=meta_len, names=columns)
    data = pd.concat([data, item], ignore_index=True)

e - elecron, Brem - тормозное излучение, Ionic - ионизационные потери (неупругое)

conv - обратная аннигилияция (фотоны в электроны и позитроны)

annihil - аннигилиция

phot - фотоэффект

In [12]:
data

,particle_name,creator_process,parentID,energy
0,e-,compt,1,12.116800
1,e-,compt,1,3.660700
2,e-,compt,1,4.600220
3,gamma,eBrem,2,0.000000
4,e-,phot,4,0.004900
...,...,...,...,...
186,e-,conv,1,5.719790
187,e-,eIoni,2,1.048250
188,e-,eIoni,16,0.525575
189,e-,compt,1,2.652640


In [19]:
set(data['parentID'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 15, 16, 19, 21}

In [14]:
filt = data[data['creator_process'] == 'conv']
# filt = filt[filt['particle_name'] == 'e+' | filt['particle_name'] == 'e-']

In [15]:
len(data[data["parentID"] == 1])

59

In [16]:
secondary = data[data["parentID"] == 1]
secondary_e_energy = sum(secondary[secondary['particle_name']=='e-']['energy'])

In [17]:
primary = data[data["parentID"] == 0]
primary_energy = sum(primary['energy'])
secondary_e_energy/primary_energy

ZeroDivisionError: float division by zero

In [ ]:
plt.hist(secondary['energy']/150000, bins=100)
plt.show()